### Registration

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'..\CCM', nargout=0)
eng.addpath(r'..\CCM\util', nargout=0)

In [2]:
import sys
sys.path.append('..')

from CCM import framework as methods
from CCM import assessment as assess

import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

# read the fixed image and mask
fixed_location = '../Data/NRS/'
fixed = sitk.ReadImage(os.path.join(fixed_location, 'fixed.mha'))
mask = sitk.ReadImage(os.path.join(fixed_location, 'mask.mha'))

# groups = range(10) # for all groups
# angles = range(0,360,10) # for all angles

groups = range(1)
angles = range(180,181)

for group in groups:
    print('group', group)
    output_location = f'../Data/NRS/NRS_Group_{group}/'
    
    for angle in angles:
        
        print('angle', angle)
        
        # read moving image
        iter_output_location = os.path.join(output_location, "Angle_%03d" % angle)
        moving = sitk.ReadImage(os.path.join(iter_output_location, 'moving.mha'))
        gt_field = sitk.ReadImage(os.path.join(iter_output_location, 'gt_field.mha'))

        print('\rCCM Flight ...', end='')
        reg = methods.Registration(fixed=fixed, moving=moving, matlab_eng=eng)
        reg.PerformCCMRegistration(
            nlevel=[2], nmigr=[1], niter=[0], step=1, sigma=16,
            feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=150, feature_detector_ratio=[4], feature_detector_num=[1000],
            feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0, 3, 5, 7, 9], feature_descriptor_samplenum=[1, 10, 15, 20, 25], feature_descriptor_dmax=5.5, feature_descriptor_dmin=8,
            feature_match_ratio=1, feature_match_crosscheck=True,
            route_merge=True, route_check=True, route_stay=0.01,
            ccm_k=[50], ccm_e=[0.5], ccm_j=[0.5], ccm_t=[0.5], ccm_z=[0.5])
        print('\rCCM Flight, done.')
        reg.OutputRegistrationResult(iter_output_location+'/ccm_flight_')
        print(assess.Assessment(registration=reg, mask=mask, gt_field=gt_field).AssessRegistration())
        
        print('\rCCM ...', end='')
        reg = methods.Registration(fixed=fixed, moving=moving, matlab_eng=eng)
        reg.PerformCCMRegistration(
            nlevel=[2,1], nmigr=[3,3], niter=[200,200], step=1, sigma=16,
            feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=150, feature_detector_ratio=[4,2], feature_detector_num=[1000,1000],
            feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0, 3, 5, 7, 9], feature_descriptor_samplenum=[1, 10, 15, 20, 25], feature_descriptor_dmax=5.5, feature_descriptor_dmin=8,
            feature_match_ratio=1, feature_match_crosscheck=True,
            route_merge=True, route_check=True, route_stay=0.01,
            ccm_k=[50,50], ccm_e=[0.5,0.5], ccm_j=[0.5,0.5], ccm_t=[0.5,0.5], ccm_z=[0.5,0.5])
        reg.OutputRegistrationResult(iter_output_location+'/ccm_')
        print('\rCCM, done.')
        print(assess.Assessment(registration=reg, mask=mask, gt_field=gt_field).AssessRegistration())

c:\Users\Administrator\Documents\Envs\crowd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


group 0
angle 180
CCM Flight, done.
{'mse': [701.5250774801469, 2201.202455185283, 112675], 'jac': [0.9756982322156134, 0.1696927880079139, 112675], 'tre': [-1, -1, -1], 'err': [4.244557703351208, 5.773378491860486, 112675], 'SDlogJ': 0.3414468920831963, 'dice': [], 'hdd': [], 'time': [1.801011085510254]}
CCM, done.
{'mse': [52.012165574065726, 170.46760645399294, 112675], 'jac': [1.0003747251170796, 0.12358392880935212, 112675], 'tre': [-1, -1, -1], 'err': [1.2784170282369107, 2.112289989519708, 112675], 'SDlogJ': 0.2928103874778229, 'dice': [], 'hdd': [], 'time': [4.318360328674316]}
